<a href="https://colab.research.google.com/github/LukyLuke92/freeCodeCamp-files-LR/blob/main/MachineLearningWithPython/LinearRegressionHealthCosts/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Take a look at the data
dataset['region'].value_counts()

In [ ]:
# Converters for categorical variables
# The more efficient way to do this is to create a set from each dataframe,
#  then assign a different int to each unique value - may come back and do this
# For sex (male = 0, female = 1)
def sex_to_int(sex):
  if sex == 'male':
    return 0
  elif sex == 'female':
    return 1
  elif isinstance(sex,int):
    return sex

# For smoker (no = 0, yes = 1)
def smoker_to_int(smoker):
  if smoker == 'no':
    return 0
  elif smoker == 'yes':
    return 1
  elif isinstance(smoker,int):
    return smoker

In [ ]:
# Because region does not have a binary variable, the best way to do this would
#  be to split up the regions into separate variables and have those be binary
for i in ['southeast','southwest','northeast','northwest']:
  dataset[i] = dataset['region'] == i
  dataset[i] = dataset[i].astype('int64')
dataset = dataset.drop(columns='region')

In [ ]:
# Use the converters above to convert categorical fields to ints
dataset['sex'] = dataset['sex'].apply(lambda x: sex_to_int(x))
dataset['smoker'] = dataset['smoker'].apply(lambda x: smoker_to_int(x))
print(dataset)

In [ ]:
# Split the dataset into train and test datasets
split = int(0.8*dataset.shape[0])
train_dataset = dataset.iloc[:split]
test_dataset = dataset.iloc[split:]
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')
print(train_dataset)

In [ ]:
# To create a linear regression, make a sequential model with an input layer,
#  no hidden layers, and a single output node - this creates a model where all
#  of the inputs are added with separate weights to yield an output (which is a
#  linear model)
# Added a 'hidden layer' that simply adds a bias, which is just a constant that
#  will be added (essentially the intercept) - this seems to improve the MAE
#  quite a bit
batch_size = 64

model = keras.Sequential([
    keras.Input(shape=(None,9),
                batch_size=batch_size
                ),
    layers.Dense(1,use_bias=True),
    layers.Dense(1)
])
# Code below uses a normalization pre-processing layer instead - both appear
#  to work well, but wanted to use something other than what is used in the tf
#  fuel efficiency tutorial
'''
normalize = layers.Normalization(axis=-1)
normalize.adapt(train_dataset)
model = keras.Sequential([
    normalize,
    #layers.Dense(1,use_bias=True),
    layers.Dense(1)
])
'''

In [ ]:
# Compile the model
# Using mean_absolute_error, since it is written below that mean absolute error
#  is being evaluated
# Previously tried to use default rmsprop, but increasing the learning rate
#  appears to make the model train much MUCH faster
# Set metrics=['mae','mse'] so that it will return the metrics (along with loss
#  function value) when model.evaluate is called later
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1), #'rmsprop',
              loss='mean_absolute_error',
              metrics=['MAE','MSE']
              )

In [ ]:
# Train the model using model.fit()
# MAE appeared to still be decreasing after 100 epochs
history = model.fit(x=train_dataset,
          y=train_labels,
          epochs=200,
          validation_split=0.2
          )

In [ ]:
# Plot how the loss function is changing over time
hist = pd.DataFrame(history.history)
hist.loss.plot()
hist.val_loss.plot()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
